In [2]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [3]:
token='vk1.a.MJEXEuq2lLn8WH4JG-Ak63reniK6DC61uLEdl7urJ1YT87pgHCT74AxwhAzt3l_4yNDm1GcwghxrZnYUkguFJmCS0H7GXUBCs7BBT-AQ8N5XL3khFo38zCoWEi3KsUZ_GNKe1T6P0fpQNSsQI7NazIYoyujI9k2QtLGFFEoTZvNuVPt5kDgB2dVi_Kn5GzQnIzCG0PjMIrtp7NHu4yJrXA'

In [5]:
vk_session = vk_api.VkApi (token=token)
vk = vk_session.get_api()

In [9]:
vk.messages.send(
    chat_id=1,
    random_id=2,
    message='Wake up, Samurai....')

0

In [11]:
df= pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/shared/ads_data.csv.zip', compression='zip')

In [12]:
ad_data=df.groupby(['ad_id','event'],as_index=False)\
    .agg({'user_id':'count'})

In [16]:
ad_data=ad_data.pivot(index='ad_id',columns='event',values='user_id').reset_index()

In [18]:
ad_data = ad_data.fillna(0).assign(ctr=ad_data.click/ad_data.view)

In [22]:
top_ctr=ad_data.query('click>20 & view>100 ').sort_values('ctr').tail(10)

In [24]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [26]:
path='/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/Lesson_7/top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file=path
upload_url=vk.docs.getMessagesUploadServer(peer_id=2000000001)["upload_url"]
file ={'file':(file_name,open(path_to_file,'rb'))}

response=requests.post(upload_url,files=file)

json_data=json.loads(response.text)
json_data

{'file': '250765353|0|-1|909418|4dc3d52feb|csv|403|top_ctr_data.csv|c2f9e346fc49058f43500d24dbf7ae44|9fdd8b7d75af1567f6349b7755445c93||||eyJkaXNrIjoiMjMiLCJwZWVyX3NlbmRlciI6Ii0yMjM0NzUwMjEifQ=='}

In [27]:
saved_file=vk.docs.save(file=json_data['file'],title=file_name)

In [28]:
saved_file

{'type': 'doc',
 'doc': {'id': 669208127,
  'owner_id': 250765353,
  'title': 'top_ctr_data.csv',
  'size': 403,
  'ext': 'csv',
  'date': 1700148057,
  'type': 1,
  'url': 'https://vk.com/doc250765353_669208127?hash=K0WgssZeldppdcn4wMnu1VHiZgPNuWDazi8xJi7EEKz&dl=Jaukqcx7qHibbApvsYSMRKQyWMRkUmZN1lAOCKIsamP&api=1&no_preview=1',
  'is_unsafe': 0}}

In [29]:
attachment='doc{}_{}'.format(saved_file['doc']['owner_id'],saved_file['doc']['id'])

In [30]:
attachment

'doc250765353_669208127'

In [31]:
vk.messages.send(
    chat_id=1,
    random_id=3,
    message='Топ объявлений по CTR',
    attachment=attachment
)

0

In [32]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

my_mail = 'alexzakorko@gmail.com'

# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name('heroic-venture-268009-1dfbcc34e5fa.json', scope)
gs = gspread.authorize(credentials)